In [71]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
import numpy as np
from nltk import word_tokenize, FreqDist
import string
from os import path
from PIL import Image
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator
# from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer as sa
# from nltk.classify.scikitlearn import SklearnClassifier
# from nltk.classify import ClassifierI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.model_selection import train_test_split, cross_validate
from surprise import Dataset, Reader, accuracy, SVD, NMF
reader= Reader(rating_scale=(1, 5.0))
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Trainset
import requests
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity


with open('userratingsDF.pickle', 'rb') as f:
    dfr = pickle.load(f)
with open('CategoricalItemDF.pickle', 'rb') as f:
    full = pickle.load(f)
with open('_SENT&RATINGDF.pickle', 'rb') as f:
    df1 = pickle.load(f)
with open('recdf.pickle', 'rb') as f:
    rec = pickle.load(f)
with open('ratingsmeandf.pickle', 'rb') as f:
    ratemean = pickle.load(f)
with open('newratingsDFwithimg.pickle', 'rb') as f:
    images = pickle.load(f)

In [98]:
with open('findf.pickle', 'rb') as f:
    rec = pickle.load(f)

In [99]:
rec

,price,prodName,rating,url,image,age,darkcircles,acne,dry,redness,...,lotion,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool
0,29.99,La Roche-Posay Effaclar Adapalene Gel 0.1% Ret...,4.91,12190079,https://s1.thcdn.com/productimg/600/600/121900...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,29.50,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,https://s1.thcdn.com/productimg/600/600/111741...,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,85.00,Slip Silk Pillowcase - Queen - White,4.68,11561755,https://s1.thcdn.com/productimg/600/600/115617...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,83.00,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,3.00,11289619,https://s1.thcdn.com/productimg/600/600/112896...,1,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,28.00,ARCONA MEN Clarity Aftershave Pads (45 Pads),4.86,11286252,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,47.00,ARCONA Cranberry Gommage 3.4oz,5.00,11286233,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,85.00,Slip Silk Pillowcase - Queen - Pink,4.11,11561754,https://s1.thcdn.com/productimg/600/600/115617...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
7,38.00,ARCONA Cranberry Toner 4.05oz,4.11,11286235,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,67.00,SkinCeuticals Phyto Corrective Gel,5.00,11289629,https://s1.thcdn.com/productimg/600/600/112896...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,435.00,Dr Dennis Gross Skincare DRx FaceWare Pro Dome...,4.00,11931276,https://s1.thcdn.com/productimg/600/600/119312...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1


# Beginning Recommender System Using Just UserId, ItemID, and Rating

#### Item Based Collaborative Filtering probably better than User Based bc we have more users than items

In [73]:
toadd=full[['url','prodName','age','darkcircles','dry','redness','sensitive', 'oily', 'normal', 'cleanser',
       'exfoliator', 'makeup-removers', 'toner', 'mist', 'treatment', 'serum',
       'lotion', 'moisturizer', 'balm', 'oil', 'mask', 'peel', 'lip', 'eye',
       'supplement', 'tool']]

In [74]:
rec=rec.merge(toadd,on='url').drop_duplicates()
rec=rec.merge(ratemean,on='url').drop_duplicates()
rec.rename(columns={'rating_x':'rating','prodName_x':'brandName','prodName_y':'prodName'},inplace=True)
rec.drop(columns='rating_y',inplace=True)
# rec['image']=images['image']
# for ur, ui in zip(rec.url,images.url):
#     while ur==ui:
#         rec['image']=images[images['url']==ui]['image']
# rec['image']=images['image']
# rec['image']

In [79]:
rec

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
5,Paula's Choice,5,11174178,tilly,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
6,Paula's Choice,5,11174178,anne206,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
7,Paula's Choice,5,11174178,samsara33139,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
8,Paula's Choice,5,11174178,suzi,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
9,SkinCeuticals,5,11289629,kiko,SkinCeuticals Phyto Corrective Gel,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,70


In [83]:
test=rec[rec['normal']==1]

In [84]:
test

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
5,Paula's Choice,5,11174178,tilly,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
6,Paula's Choice,5,11174178,anne206,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
7,Paula's Choice,5,11174178,samsara33139,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
8,Paula's Choice,5,11174178,suzi,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
9,SkinCeuticals,5,11289629,kiko,SkinCeuticals Phyto Corrective Gel,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,70


In [90]:
test=rec[rec['normal']==1]
stype=['dry','tool']
for s in stype:
    new=rec[rec[s]==1]
    test=pd.concat([test,new])

In [94]:
stype=['dry','tool','sensitive']

In [96]:
#rec[rec['normal']==1].append(rec[rec['dry']==1]) 
df=rec[rec['normal']==1]
for s in stype:
    df=df.append(rec[rec[s]==1])
df.drop_duplicates(inplace=True)

In [97]:
df

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
5,Paula's Choice,5,11174178,tilly,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
6,Paula's Choice,5,11174178,anne206,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
7,Paula's Choice,5,11174178,samsara33139,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
8,Paula's Choice,5,11174178,suzi,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
9,SkinCeuticals,5,11289629,kiko,SkinCeuticals Phyto Corrective Gel,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,70


In [50]:
with open('df.pickle', 'wb') as f:
    pickle.dump(rec, f, pickle.HIGHEST_PROTOCOL)

In [37]:
!pip install dash_package

  Could not find a version that satisfies the requirement dash_package (from versions: )
No matching distribution found for dash_package


In [9]:
data = Dataset.load_from_df(rec[['user','url','rating']],reader)

In [10]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  1378 

Number of items:  514


## Trying to Minimize RMSE

#### SVD Grid Search

In [9]:
params = {'n_factors' :[10,20,30,40,50,60,70,80,90,100,200],
         'reg_all':[0.02,0.05,0.1,0.4,0.5,0.6],'n_epochs': [5, 10],
         'lr_all': [0.002, 0.005],'reg_all': [0.4, 0.6]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [10]:
print(g_s_svd.best_score['rmse'])
print(g_s_svd.best_params['rmse'])

0.9598922333672055
{'n_factors': 30, 'reg_all': 0.4, 'n_epochs': 10, 'lr_all': 0.005}


#### NMF Grid Search

In [11]:
params = {'n_factors' :[5,10,20,30,40,50,60,70,80,90,100,200],
          'n_epochs': [5, 10]}
g_s_nmf = GridSearchCV(NMF,param_grid=params,n_jobs=-1)
g_s_nmf.fit(data)

In [12]:
print(g_s_nmf.best_score['rmse'])
print(g_s_nmf.best_params['rmse'])

0.992410211663158
{'n_factors': 200, 'n_epochs': 10}


## KNN

### SIM_OPTIONS: Cosine Item Based

#### KNN Basic

In [10]:
knn_basic = KNNBasic(sim_options={'name':'cosine','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.90890161, 0.96873017, 1.04439869, 1.07426939, 0.88591421]))
('test_mae', array([0.65695256, 0.66090125, 0.71800262, 0.71289308, 0.63166706]))
('fit_time', (0.007392168045043945, 0.006783962249755859, 0.003638029098510742, 0.003305196762084961, 0.003809213638305664))
('test_time', (0.002286195755004883, 0.0019850730895996094, 0.0016338825225830078, 0.0016298294067382812, 0.0017268657684326172))
-----------------------
0.9764428139415496


#### KNN Baseline

In [11]:
knn_baseline = KNNBaseline(sim_options={'name':'cosine','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.99612817, 0.95286665, 0.88321094, 1.04089497, 0.97620286]))
('test_mae', array([0.67881961, 0.6670182 , 0.63137781, 0.69682684, 0.68163483]))
('fit_time', (0.010792970657348633, 0.013057947158813477, 0.012003183364868164, 0.008646249771118164, 0.009865045547485352))
('test_time', (0.0023720264434814453, 0.0022149085998535156, 0.001962900161743164, 0.0017931461334228516, 0.0020279884338378906))
-----------------------
0.9698607156487693


#### KNN With Means

In [12]:
knn_means = KNNWithMeans(sim_options={'name':'cosine','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.98928253, 0.96386456, 1.08176763, 0.93471887, 1.04349242]))
('test_mae', array([0.67594669, 0.65073787, 0.71125445, 0.62537662, 0.693286  ]))
('fit_time', (0.009605884552001953, 0.014175891876220703, 0.009071111679077148, 0.00913095474243164, 0.009814977645874023))
('test_time', (0.0024759769439697266, 0.0023469924926757812, 0.0019469261169433594, 0.0019519329071044922, 0.001940011978149414))
-----------------------
1.002625201350422


### SIM_OPTIONS: Cosine USer Based

#### KNN Basic

In [13]:
knn_basic = KNNBasic(sim_options={'name':'cosine','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.02378459, 0.98826338, 0.92019561, 0.92821787, 1.03198381]))
('test_mae', array([0.69852361, 0.6795832 , 0.64940254, 0.66296076, 0.69901216]))
('fit_time', (0.039000749588012695, 0.038290977478027344, 0.022185802459716797, 0.021097898483276367, 0.03825712203979492))
('test_time', (0.0021789073944091797, 0.002238035202026367, 0.0019567012786865234, 0.0017437934875488281, 0.0020956993103027344))
-----------------------
0.9784890507476692


#### KNN Baseline

In [14]:
knn_baseline = KNNBaseline(sim_options={'name':'cosine','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.95322255, 0.94008821, 0.99488813, 1.01733195, 0.94909613]))
('test_mae', array([0.67528282, 0.65399087, 0.66775162, 0.69638874, 0.66794032]))
('fit_time', (0.04721879959106445, 0.04445219039916992, 0.027058124542236328, 0.029604196548461914, 0.02437305450439453))
('test_time', (0.002094268798828125, 0.002009868621826172, 0.0019578933715820312, 0.0019707679748535156, 0.0019328594207763672))
-----------------------
0.9709253923312294


#### KNN With Means

In [15]:
knn_means = KNNWithMeans(sim_options={'name':'cosine','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.09036891, 1.11641172, 0.92886349, 0.97348824, 1.04511284]))
('test_mae', array([0.72691443, 0.70942666, 0.62237949, 0.64560923, 0.68449777]))
('fit_time', (0.03462100028991699, 0.050513267517089844, 0.030248403549194336, 0.029983043670654297, 0.029340744018554688))
('test_time', (0.0022242069244384766, 0.002112865447998047, 0.0019447803497314453, 0.0018761157989501953, 0.0018050670623779297))
-----------------------
1.0308490381068518


### SIM_OPTIONS Peasrson Item Based

#### KNN Basic

In [16]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97130197, 0.81901005, 0.95187453, 0.93549939, 1.18820035]))
('test_mae', array([0.67904909, 0.61390692, 0.66777224, 0.66599735, 0.76904873]))
('fit_time', (0.009018898010253906, 0.008264780044555664, 0.008024215698242188, 0.004899024963378906, 0.008108139038085938))
('test_time', (0.0019428730010986328, 0.0019230842590332031, 0.0017616748809814453, 0.0016171932220458984, 0.001622915267944336))
-----------------------
0.9731772572474464


#### KNN Baseline

In [17]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.99041302, 0.92218288, 1.04242295, 0.9810611 , 0.88106264]))
('test_mae', array([0.68753669, 0.65733102, 0.69940034, 0.6703398 , 0.62661801]))
('fit_time', (0.011060953140258789, 0.012090682983398438, 0.013730049133300781, 0.010601043701171875, 0.009036064147949219))
('test_time', (0.002457857131958008, 0.0021610260009765625, 0.0019049644470214844, 0.0018520355224609375, 0.0018849372863769531))
-----------------------
0.96342851812440

#### KNN With Means

In [18]:
knn_means = KNNWithMeans(sim_options={'name':'pearson','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.81132872, 0.98468451, 1.18555963, 1.09383117, 0.8860752 ]))
('test_mae', array([0.59433053, 0.65694103, 0.76811823, 0.71201119, 0.62111259]))
('fit_time', (0.011271953582763672, 0.013571023941040039, 0.009980916976928711, 0.009370803833007812, 0.009818792343139648))
('test_time', (0.002554178237915039, 0.001990795135498047, 0.0018990039825439453, 0.0019109249114990234, 0.0018329620361328125))
-----------------------
0.9922958470326719


### SIM_OPTIONS Pearson User Based

#### KNN Basic

In [19]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.04782399, 0.93931161, 0.99231992, 0.962505  , 0.9496872 ]))
('test_mae', array([0.70517946, 0.66489476, 0.68739019, 0.6717523 , 0.66395353]))
('fit_time', (0.050451040267944336, 0.05025315284729004, 0.050475120544433594, 0.05158710479736328, 0.05046200752258301))
('test_time', (0.0021250247955322266, 0.0021309852600097656, 0.0021729469299316406, 0.002089977264404297, 0.0020711421966552734))
-----------------------
0.97832954532504


#### KNN Basline

In [20]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.84693099, 0.97838634, 0.91112664, 1.00209625, 1.08094817]))
('test_mae', array([0.61316586, 0.67859862, 0.6457759 , 0.68476307, 0.71086616]))
('fit_time', (0.04794597625732422, 0.0300750732421875, 0.03400993347167969, 0.03227591514587402, 0.029616832733154297))
('test_time', (0.0021009445190429688, 0.001970052719116211, 0.0019719600677490234, 0.0019750595092773438, 0.0018768310546875))
-----------------------
0.963897676669365


#### KNN With Means

In [21]:
knn_means = KNNWithMeans(sim_options={'name':'pearson','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.93046592, 1.0244918 , 0.99565652, 1.14708287, 1.04244671]))
('test_mae', array([0.64010452, 0.68063349, 0.67257611, 0.73121138, 0.67686857]))
('fit_time', (0.04549407958984375, 0.03548288345336914, 0.03435111045837402, 0.038459062576293945, 0.03566288948059082))
('test_time', (0.0021600723266601562, 0.0020029544830322266, 0.002035856246948242, 0.002042055130004883, 0.002023935317993164))
-----------------------
1.0280287634612058


### SIM_OPTIONS MSD Item Based

#### KNN Basic

In [22]:
knn_basic = KNNBasic(sim_options={'name':'msd','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.98155994, 1.05738169, 0.83585361, 0.99638405, 1.01628067]))
('test_mae', array([0.67613575, 0.70601761, 0.62672559, 0.68500125, 0.69301965]))
('fit_time', (0.0042591094970703125, 0.0024280548095703125, 0.0024640560150146484, 0.0025670528411865234, 0.004168987274169922))
('test_time', (0.0019698143005371094, 0.001802206039428711, 0.0016589164733886719, 0.0017819404602050781, 0.0015718936920166016))
-----------------------
0.9774919905189868


#### KNN Baseline

In [23]:
knn_baseline = KNNBaseline(sim_options={'name':'msd','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.85083688, 0.99753122, 0.99528502, 1.01157128, 0.97937054]))
('test_mae', array([0.61473764, 0.67668714, 0.68802986, 0.69778857, 0.68269928]))
('fit_time', (0.008715629577636719, 0.008648872375488281, 0.008142948150634766, 0.007617950439453125, 0.009401082992553711))
('test_time', (0.00234222412109375, 0.0022351741790771484, 0.0019693374633789062, 0.0019011497497558594, 0.002317667007446289))
-----------------------
0.9669189870383536


#### KNN With Means

In [24]:
knn_means = KNNWithMeans(sim_options={'name':'msd','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.96114978, 1.0959542 , 1.01024245, 0.94865695, 0.96314814]))
('test_mae', array([0.65035364, 0.7336231 , 0.66529959, 0.64422875, 0.65188329]))
('fit_time', (0.008625268936157227, 0.008409976959228516, 0.007506847381591797, 0.008149862289428711, 0.00710296630859375))
('test_time', (0.002794981002807617, 0.002241849899291992, 0.0021660327911376953, 0.0019021034240722656, 0.0018150806427001953))
-----------------------
0.9958303058431325


### SIM_OPTIONS MSD USER BASED

#### KNN Basic

In [25]:
knn_basic = KNNBasic(sim_options={'name':'msd','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.92620719, 0.97582947, 0.89887092, 1.10012583, 0.97678743]))
('test_mae', array([0.6470517 , 0.68238672, 0.6337669 , 0.74115018, 0.67654362]))
('fit_time', (0.02525496482849121, 0.024525165557861328, 0.025012969970703125, 0.014275789260864258, 0.013466835021972656))
('test_time', (0.001978158950805664, 0.0018057823181152344, 0.0017578601837158203, 0.001764059066772461, 0.001683950424194336))
-----------------------
0.9755641686421637


#### KNN Baseline

In [26]:
knn_baseline = KNNBaseline(sim_options={'name':'msd','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.97117462, 1.00444676, 0.98017933, 0.92747982, 0.93533296]))
('test_mae', array([0.67551955, 0.67448547, 0.67546748, 0.65381205, 0.65376935]))
('fit_time', (0.0247650146484375, 0.024112939834594727, 0.023098230361938477, 0.021278858184814453, 0.02052783966064453))
('test_time', (0.0021719932556152344, 0.0020630359649658203, 0.0020439624786376953, 0.0019588470458984375, 0.0018610954284667969))
-----------------------
0.9637226971319389


#### KNN With Means

In [27]:
knn_means = KNNWithMeans(sim_options={'name':'msd','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.0650466 , 1.03268332, 0.91217639, 1.06779803, 1.03401292]))
('test_mae', array([0.7031207 , 0.69645738, 0.63011552, 0.67766264, 0.6747759 ]))
('fit_time', (0.03048086166381836, 0.028974056243896484, 0.025653839111328125, 0.03713202476501465, 0.024395227432250977))
('test_time', (0.0022459030151367188, 0.002057790756225586, 0.0017948150634765625, 0.0019600391387939453, 0.0018177032470703125))
-----------------------
1.0223434538181515


### SIM_OPTIONS Pearson Baseline Item Based

#### KNN Basic

In [28]:
knn_basic = KNNBasic(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.0560139 , 1.00123293, 0.98362094, 0.88711643, 0.96349985]))
('test_mae', array([0.72215748, 0.67904012, 0.68011582, 0.63937843, 0.67395747]))
('fit_time', (0.008800029754638672, 0.009472846984863281, 0.007248878479003906, 0.008254289627075195, 0.008337974548339844))
('test_time', (0.0016901493072509766, 0.0015530586242675781, 0.0014579296112060547, 0.0015668869018554688, 0.0013470649719238281))
-----------------------
0.9782968093238683


### ** KNN Baseline !!!! BEST RMSE

In [29]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.01631249, 1.03736203, 0.94485934, 0.77454699, 1.04143234]))
('test_mae', array([0.68981636, 0.70155923, 0.65558639, 0.59135392, 0.70446414]))
('fit_time', (0.009500265121459961, 0.01034998893737793, 0.008582830429077148, 0.00843501091003418, 0.008971929550170898))
('test_time', (0.002313852310180664, 0.002195119857788086, 0.001964092254638672, 0.0018458366394042969, 0.0018301010131835938))

#### KNN With Means

In [30]:
knn_means = KNNWithMeans(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.95872854, 1.04504525, 0.85829199, 1.00630934, 1.11081022]))
('test_mae', array([0.65255284, 0.68634423, 0.6156029 , 0.66619378, 0.71975712]))
('fit_time', (0.015146970748901367, 0.013919830322265625, 0.0140380859375, 0.011783123016357422, 0.011476993560791016))
('test_time', (0.0024759769439697266, 0.0021729469299316406, 0.0016109943389892578, 0.0015730857849121094, 0.0015060901641845703))

### SIM_OPTIONS Pearson Baseline User Based

#### KNN Basic

In [31]:
knn_basic = KNNBasic(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.00660741, 1.02914153, 0.93523495, 0.97086927, 0.95297836]))
('test_mae', array([0.68433791, 0.712037  , 0.65650436, 0.67643157, 0.66652175]))
('fit_time', (0.037883758544921875, 0.03819990158081055, 0.04076385498046875, 0.03890585899353027, 0.025016069412231445))
('test_time', (0.001699209213256836, 0.0016701221466064453, 0.0017249584197998047, 0.001873016357421875, 0.00185394287109375))
-----------------------
0.9789663035452854


#### KNN Baseline

In [32]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.93507832, 0.94977475, 0.92931432, 1.03253165, 0.97293754]))
('test_mae', array([0.6581214 , 0.65803357, 0.64492564, 0.69885185, 0.6732842 ]))
('fit_time', (0.02298426628112793, 0.026391983032226562, 0.02531886100769043, 0.024007797241210938, 0.0222930908203125))
('test_time', (0.0021140575408935547, 0.0020360946655273438, 0.0018177032470703125, 0.0019609928131103516, 0.0018601417541503906)

#### KNN With Means

In [33]:
knn_means = KNNWithMeans(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.15973383, 1.08876134, 1.06731528, 0.91843529, 0.84126561]))
('test_mae', array([0.74402169, 0.71489401, 0.69256935, 0.61275548, 0.60107076]))
('fit_time', (0.037213802337646484, 0.03287768363952637, 0.03197789192199707, 0.030858278274536133, 0.033930063247680664))
('test_time', (0.0018820762634277344, 0.001683950424194336, 0.0017039775848388672, 0.0016200542449951172, 0.0016276836395263672

# Collaborative Filtering, Item Based

## Trying to Predict for New Users

In [92]:
df=rec[rec['normal']==1]
df2=rec[rec['dry']==1]
pd.concat([df,df2]).drop_duplicates()

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
5,Paula's Choice,5,11174178,tilly,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
6,Paula's Choice,5,11174178,anne206,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
7,Paula's Choice,5,11174178,samsara33139,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
8,Paula's Choice,5,11174178,suzi,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
9,SkinCeuticals,5,11289629,kiko,SkinCeuticals Phyto Corrective Gel,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,70


In [13]:
from PIL import Image
import requests
from io import BytesIO



In [44]:
rec['image'][0]

's1.thcdn.com/productimg/600/600/11174178-2034649963595204.jpg'

In [47]:
rec.sample(1)['image'].item()

's1.thcdn.com/productimg/600/600/11363395-1844508710845681.jpg'

In [88]:
p = rec.sample(1)

In [72]:
p['image'].item()

's1.thcdn.com/productimg/0/600/600/87/10915787-1392369363-6261'

In [62]:
rec.sample(3)

,prodName,rating,url,user
2979,Dr. Brandt,5,10543756,sugarface
1023,ARCONA,5,11286282,eaf1
2297,Alchimie Forever,5,11296164,fm


In [78]:
rec

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
5,Paula's Choice,5,11174178,tilly,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
6,Paula's Choice,5,11174178,anne206,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
7,Paula's Choice,5,11174178,samsara33139,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
8,Paula's Choice,5,11174178,suzi,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
9,SkinCeuticals,5,11289629,kiko,SkinCeuticals Phyto Corrective Gel,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,70


# The Wrong URLs are showing up :/

In [67]:
def skin_rater(movie_df,num, typ=None):
    rating_list = []
    knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':False})
    #Start with 'normal/combination skin df'
    df=movie_df.loc[movie_df['normal']==1].copy()
    raw_uid=input('type username: ')
    #filter df by preferences
    stype=input('What is your skin type or skin problems? Type all that apply: dry, sensitive, oily, redness, dark circles, aging. (n if none apply) ')
    ptype=input('Are you looking for any of these products? \ncleanser \nexfoliator\n makeup-remover\n toner \nmist \ntreatment \nserum \n lotion \nmoisturizer \nbalm \noil \nmask \npeel \nlip \neye \nsupplement \ntool ')
    if 'dry' in stype:
        df=pd.concat([df,movie_df[movie_df['dry']==1]])
    if 'aging' in stype:
        df=pd.concat([df,movie_df[movie_df['age']==1]])
    if 'dark circles' in stype:
        df=pd.concat([df,movie_df[movie_df['darkcircles']==1]])
    if 'redness' in stype:
        df=pd.concat([df,movie_df[movie_df['redness']==1]])
    if 'sensitive' in stype:
        df=pd.concat([df,movie_df[movie_df['sensitive']==1]])
    if 'oily' in stype:
        df=pd.concat([df,movie_df[movie_df['oily']==1]])
    if 'cleanser' in ptype:
        df=pd.concat([df,movie_df[movie_df['cleanser']==1]])
    if 'exfoliator' in ptype:
        df=pd.concat([df,movie_df[movie_df['exfoliator']==1]])
    if 'makeup-remover' in ptype:
        df=pd.concat([df,movie_df[movie_df['makeup-removers']==1]])
    if 'toner' in ptype:
        df=pd.concat([df,movie_df[movie_df['toner']==1]])
    if 'mist' in ptype:
        df=pd.concat([df,movie_df[movie_df['mist']==1]])
    if 'treatment' in ptype:
        df=pd.concat([df,movie_df[movie_df['treatment']==1]])
    if 'serum' in ptype:
        df=pd.concat([df,movie_df[movie_df['serum']==1]])
    if 'lotion' in ptype:
        df=pd.concat([df,movie_df[movie_df['lotion']==1]])
    if 'moisturizer' in ptype:
        df=pd.concat([df,movie_df[movie_df['moisturizer']==1]])
    if 'balm' in ptype:
        df=pd.concat([df,movie_df[movie_df['balm']==1]])
    if 'oil' in ptype:
        df=pd.concat([df,movie_df[movie_df['oil']==1]])
    if 'mask' in ptype:
        df=pd.concat([df,movie_df[movie_df['mask']==1]])
    if 'peel' in ptype:
        df=pd.concat([df,movie_df[movie_df['peel']==1]])
    if 'lip' in ptype:
        df=pd.concat([df,movie_df[movie_df['lip']==1]])
    if 'eye' in ptype:
        df=pd.concat([df,movie_df[movie_df['eye']==1]])
    if 'supplement' in ptype:
        df=pd.concat([df,movie_df[movie_df['supplement']==1]])
    if 'tool' in ptype:
        df=pd.concat([df,movie_df[movie_df['tool']==1]])
    df.drop_duplicates(inplace=True)
    #Asking User to rate these products from their preferences
    while num > 0:
        p = df.sample(1)
#         response = requests.get('https://' + p['image'].item())
#         img = Image.open(BytesIO(response.content))
#         print(p[['brandName','prodName','url']],'https://' + str(p['image']))
        print(p[['brandName','prodName','url']])
        rating = input('How do you rate this product on a scale of 1-5, press n if you have not used :\n')
        if rating == 'n':
            continue
        #Keeping Track of the new user's ratings
        else:
            rating_one_movie = {'user':raw_uid,'url':p['url'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    new_ratings_df = df[['user','url','rating']].append(rating_list,ignore_index=True)
    new_data = Dataset.load_from_df(new_ratings_df,reader)
    train, test=train_test_split(new_data, test_size=0.2, random_state=42, shuffle=True)
    knn_baseline.fit(train)
    #preds=knn_baseline.test(test)
    
    list_of_prods=[]
    for u in new_ratings_df['url'].unique():
        list_of_prods.append((u,knn_baseline.predict(raw_uid,u)[3]))
    ranked_prods = sorted(list_of_prods,key=lambda x:x[1],reverse=True)
    
    n=int(input('How many products are you looking for?\n'))
    for idx, re in enumerate(ranked_prods[2:]):
        title = df.loc[df['url'] == int(re[0])]['prodName']
        u=re[0]
        print('Recommendation # ',idx+1,'|| ', 'url:', u, 'image: ', '||','product: ', df[df['url']==u]['prodName'].drop_duplicates(),'\n')
        n-= 1
        if n == 0:
            break
        return ranked_prods
#     return(rating_list,raw_uid,df)

# r,raw_uid,df=skin_rater(rec,3)

# knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':False})
# new_ratings_df = df[['user','url','rating']].append(r,ignore_index=True)
# new_data = Dataset.load_from_df(new_ratings_df,reader)
# train, test=train_test_split(new_data, test_size=0.2, random_state=42, shuffle=True)
# knn_baseline.fit(train)
# preds=knn_baseline.test(test)
# print(accuracy.rmse(preds))

In [75]:
ranked_prods=skin_rater(rec,3)

type username: mack
What is your skin type or skin problems? Type all that apply: dry, sensitive, oily, redness, dark circles, aging. (n if none apply) oily
Are you looking for any of these products? 
cleanser 
exfoliator
 makeup-remover
 toner 
mist 
treatment 
serum 
 lotion 
moisturizer 
balm 
oil 
mask 
peel 
lip 
eye 
supplement 
tool tool
     brandName                       prodName       url
1542     Obagi  Obagi Nu-Derm Gentle Cleanser  11291540
How do you rate this product on a scale of 1-5, press n if you have not used :
5
     brandName                    prodName       url
1250   GlyDerm  GlyDerm Face Lotion Lite 5  11291933
How do you rate this product on a scale of 1-5, press n if you have not used :
5
         brandName                                           prodName  \
321  SkinCeuticals  SkinCeuticals Phloretin CF with Ferulic Acid V...   

          url  
321  11289626  
How do you rate this product on a scale of 1-5, press n if you have not used :
5
Estimating bi

In [77]:
ranked_prods[0][0]

'11289626'

In [53]:
full

,price,prodName,rating,url,age,darkcircles,acne,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,brand
0,25.08,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,The Ordinary
1,67.00,SkinCeuticals Phyto Corrective Gel,4.11,11289629,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,Erno Laszlo
2,105.00,Sunday Riley Good Genes Treatment 30ml,4.73,10915787,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,Slip
3,17.85,Paula's Choice Skin Balancing Pore-Reducing To...,5.00,11174171,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Slip
4,62.00,Dermalogica Special Cleansing Gel 16.9oz,4.50,10364095,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Paula's Choice
5,83.00,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,4.68,11289619,1,0,1,1,0,1,...,1,0,0,0,0,0,0,0,0,Epionce
6,98.00,SkinCeuticals Discoloration Defense Dark Spot ...,5.00,11679046,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Erno Laszlo
7,85.00,Slip Silk Pillowcase - Queen - Pink,5.00,11561754,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,Slip
8,249.00,SkinCeuticals Restore and Hydrate Regimen,5.00,11976675,1,0,1,1,0,1,...,1,0,0,0,0,0,0,0,0,ARgENTUM
9,85.00,Slip Silk Pillowcase - Queen - White,4.67,11561755,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,Slip


In [69]:
skin_rater(rec,3)

KeyError: 'normal'

In [10]:
# def rank_prods_for_user(df,raw_uid):
#     list_of_prods=[]
#     for u in df['url'].unique():
#         list_of_prods.append((u,knn_baseline.predict(raw_uid,u)[3]))
#     ranked_prods = sorted(list_of_prods,key=lambda x:x[1],reverse=True)
#     return (ranked_prods)

# ranked=rank_prods_for_user(new_ratings_df,raw_uid)

In [12]:
# def recommended_products(user_ratings,df):
#     n=int(input('How many products are you looking for?\n'))
#     for idx, re in enumerate(user_ratings):
#         title = df.loc[df['url'] == int(re[0])]['prodName']
#         u=re[0]
#         print('Recommendation # ',idx+1,'|| ', 'url:', u, 'image: ', '||','product: ', df[df['url']==u]['prodName'].drop_duplicates(),'\n')
#         n-= 1
#         if n == 0:
#             break
# recommended_products(ranked[2:],df)

How many products are you looking for?
10
Recommendation #  1 ||  url: 10543756 image:  || product:  1561    Dr. Brandt Pores No More Pore Refiner (1oz)
Name: prodName, dtype: object 

Recommendation #  2 ||  url: 11524843 image:  || product:  1485    FOREO LUNA™ 2 for Oily Skin
Name: prodName, dtype: object 

Recommendation #  3 ||  url: 11174178 image:  || product:  0    Paula's Choice Skin Perfecting 2% BHA Liquid E...
Name: prodName, dtype: object 

Recommendation #  4 ||  url: 11289690 image:  || product:  1406    SkinMedica Ultra Sheer Moisturizer (2oz)
Name: prodName, dtype: object 

Recommendation #  5 ||  url: 11286360 image:  || product:  1061    Avène Thermal Spring Water 10.58oz
Name: prodName, dtype: object 

Recommendation #  6 ||  url: 11289657 image:  || product:  1386    SkinMedica AHA/BHA Exfoliating Cleanser (6oz)
Name: prodName, dtype: object 

Recommendation #  7 ||  url: 11524845 image:  || product:  1140    FOREO LUNA™ mini 2 - Fuchsia
Name: prodName, dtype: obje

## Description TFIDF and Cosine Similarity !?!??!?!?!??! LOOKING AT REVIEW CONTENT AND TITLES

In [148]:
df1['description']=df1['title']+df1['content']

In [171]:
df1.content[0]

"i have used this product for 4 years. easy to apply. consistent product each time you buy it. i have tried others and always come back to this eye cream. the only other that i really like is anthony's eye cream with sunscreen."

In [149]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df1['description'])

In [153]:
tfidf_matrix.shape

(1685, 39382)

In [154]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [155]:
cosine_sim

array([[1.        , 0.03513622, 0.02979474, ..., 0.00730602, 0.        ,
        0.00399307],
       [0.03513622, 1.        , 0.02854128, ..., 0.        , 0.        ,
        0.        ],
       [0.02979474, 0.02854128, 1.        , ..., 0.        , 0.        ,
        0.00425702],
       ...,
       [0.00730602, 0.        , 0.        , ..., 1.        , 0.        ,
        0.05258715],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.00154483],
       [0.00399307, 0.        , 0.00425702, ..., 0.05258715, 0.00154483,
        1.        ]])

In [164]:
def item(id):  
    return df1.loc[df1['url'] == id]['description']
# Just reads the results out of the dictionary.def 
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[item_id][:num]   
    for r in recs: 
        print("Recommended: " + item(r[1]) + " (score:" +      str(r[0]) + ")")

In [157]:
df1.reset_index(inplace=True)
products = df1['url']
indices = pd.Series(df1.index, index=df1['url'])

In [203]:
def get_recommendations(product):
    idx = indices[products]
    sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    prod_indices = [i[0] for i in sim_scores]
    return products.iloc[prod_indices]

In [204]:
get_recommendations(11429304).drop_duplicates()

1     10236357
5     10305907
7     10305956
11    10306437
15    10306558
25    10362821
28    10362879
Name: url, dtype: object

In [68]:
# recommend(10362879,5)

In [69]:
alldf=dfr[['url','prodName','rating','user']]

In [70]:
alldf.groupby(['url','user','prodName']).mean().head()

rating
url      user      prodName        
10236357 adrian    Anthony        5
         george    Anthony        5
         kimber    Anthony        5
         mike      Anthony        5
         westtexas Anthony        5

In [44]:
train, test = train_test_split(data,test_size=0.2)